Goal: determine the threshold of what it takes to be a playoff team

want to know for each season, what did the teams that made the playoffs do
then we will combine the qualifications to be a playoff team for each season to determine what our definition of a "playoff team" is

1. Separate regular season games by season
2. pivot data so that each team is an observation
    - create 4 new rows (away team winner/home team winner/away team loser/home team loser) to make more sense
    - create 2 new rows for goals scored and goals against
    - create new dataset with totals (ex team = philly, count up all the away wins for philly and put into a col, count up all the home wins for philly and put in new column)
3. add column that denotes whether a team made the playoffs

Notes:
- most seasons have 1230 games bc 82 games per team w/30 teams
- 2017/18 has 1271 games bc 82 games per team w/31 teams
- not using 2012/13 bc missing too much data
- not using 2018/19 because something is messed up w home_rink_side_start

In [245]:
import pandas as pd

In [246]:
games = pd.read_csv("data_cleaned/regular_season_games")

In [247]:
# dropping old indexes
games.drop('Unnamed: 0', axis = 1, inplace = True)

In [248]:
# creating new cols with away/home winner/loser
games['away_wins'] = games.loc[games.outcome.isin(['away win OT', 'away win REG']), 'away_team_id']
games['home_wins'] = games.loc[games.outcome.isin(['home win OT', 'home win REG']), 'home_team_id']
games['away_losses'] = games.loc[games.outcome.isin(['home win OT', 'home win REG']), 'away_team_id']
games['home_losses'] = games.loc[games.outcome.isin(['away win OT', 'away win REG']), 'home_team_id']

In [249]:
# creating df for each szn    
games_07_08 = games[games['season'] == 20072008]
games_08_09 = games[games['season'] == 20082009]
games_09_10 = games[games['season'] == 20092010]
games_10_11 = games[games['season'] == 20102011]
games_11_12 = games[games['season'] == 20112012]
games_13_14 = games[games['season'] == 20132014]
games_14_15 = games[games['season'] == 20142015]
games_15_16 = games[games['season'] == 20152016]
games_16_17 = games[games['season'] == 20162017]
games_17_18 = games[games['season'] == 20172018]

In [250]:
# importing df with team names
teams = pd.read_csv('data_cleaned/team_names.csv')

In [251]:
# getting rid of unused col
teams.drop('Unnamed: 0', axis = 1, inplace = True)

In [252]:
# creating teams df for each season
teams_07_08 = teams.copy()
teams_08_09 = teams.copy()
teams_09_10 = teams.copy()
teams_10_11 = teams.copy()
teams_11_12 = teams.copy()
teams_13_14 = teams.copy()
teams_14_15 = teams.copy()
teams_15_16 = teams.copy()
teams_16_17 = teams.copy()
teams_17_18 = teams.copy()

In [253]:
# function to add counts to new dfs
list_of_conditions = ['away_wins', 'home_wins', 'away_losses', 'home_losses']
list_of_teams = teams.team_name.values

def get_counts(games_df, team_df):
    for condition in list_of_conditions:
        for team in list_of_teams: 
            team_df.loc[team_df.team_name.isin([team]), condition] = len(games_df[games_df[condition] == team])

In [254]:
# adding counts
get_counts(games_07_08, teams_07_08)
get_counts(games_08_09, teams_08_09)
get_counts(games_09_10, teams_09_10)
get_counts(games_10_11, teams_10_11)
get_counts(games_11_12, teams_11_12)
get_counts(games_13_14, teams_13_14)
get_counts(games_14_15, teams_14_15)
get_counts(games_15_16, teams_15_16)
get_counts(games_16_17, teams_16_17)
get_counts(games_17_18, teams_17_18)

In [255]:
# function to get number of goals scored by a team
def total_goals_scored(team, games_df):
    return games_df.loc[games_df.away_team_id.isin([team]), 'away_goals'].sum() + games_df.loc[games_df.home_team_id.isin([team]), 'home_goals'].sum()

In [256]:
# function to get number of goals against a team
def total_goals_against(team, games_df):
    return games_df.loc[games_df.away_team_id.isin([team]), 'home_goals'].sum() + games_df.loc[games_df.home_team_id.isin([team]), 'away_goals'].sum()

In [264]:
# function to add total goals and goals against to teams datasets
def get_goal_counts(games_df, team_df):
    for team in list_of_teams:
        team_df.loc[team_df.team_name.isin([team]), 'total_goals_scored'] = total_goals_scored(team, games_df)
        team_df.loc[team_df.team_name.isin([team]), 'total_goals_against'] = total_goals_against(team, games_df)

In [261]:
get_goal_counts(games_07_08, teams_07_08)
get_goal_counts(games_08_09, teams_08_09)
get_goal_counts(games_09_10, teams_09_10)
get_goal_counts(games_10_11, teams_10_11)
get_goal_counts(games_11_12, teams_11_12)
get_goal_counts(games_13_14, teams_13_14)
get_goal_counts(games_14_15, teams_14_15)
get_goal_counts(games_15_16, teams_15_16)
get_goal_counts(games_16_17, teams_16_17)
get_goal_counts(games_17_18, teams_17_18)

In [336]:
#creating functions to assign 'Yes' or 'No' to made_playoffs
def make_playoff_team(list_of_teams, team_df):
    team_df.fillna(value = 'No', inplace = True)
    for team in list_of_teams: 
        team_df.loc[team_df.team_name.isin([team]), 'made_playoffs'] = 'Yes'

### lists of playoff teams
data from: https://www.hockey-reference.com/playoffs/NHL_2008.html

In [340]:
playoffs_07_08 = ['Anaheim Ducks',
'Boston Bruins',
'Calgary Flames',
'Colorado Avalanche',
'Dallas Stars',
'Detroit Red Wings',
'Minnesota Wild',
'Montreal Canadiens',
'Nashville Predators',
'New Jersey Devils',
'New York Rangers',
'Ottawa Senators',
'Philadelphia Flyers',
'Pittsburgh Penguins',
'San Jose Sharks',
'Washington Capitals']

In [341]:
playoffs_08_09 = ['Pittsburgh Penguins',
'Detroit Red Wings',
'Chicago Blackhawks',
'Carolina Hurricanes',
'Boston Bruins',
'Anaheim Ducks',
'Washington Capitals',
'Vancouver Canucks',
'New Jersey Devils',
'New York Rangers',
'Philadelphia Flyers',
'Calgary Flames',
'San Jose Sharks',
'St Louis Blues',
'Montreal Canadiens',
'Columbus Blue Jackets']

In [342]:
playoffs_09_10 = ['Chicago Blackhawks',
'Philadelphia Flyers',
'Montreal Canadiens',
'San Jose Sharks',
'Pittsburgh Penguins',
'Boston Bruins',
'Vancouver Canucks',
'Detroit Red Wings',
'Phoenix Coyotes',
'Washington Capitals',
'Ottawa Senators',
'Colorado Avalanche',
'Buffalo Sabres',
'Los Angeles Kings',
'Nashville Predators',
'New Jersey Devils']

In [343]:
playoffs_10_11 = ['San Jose Sharks',
'Boston Bruins',
'Vancouver Canucks',
'Nashville Predators',
'Washington Capitals',
'Tampa Bay Lightning',
'Philadelphia Flyers',
'Pittsburgh Penguins',
'Chicago Blackhawks',
'Buffalo Sabres',
'Detroit Red Wings',
'Montreal Canadiens',
'Anaheim Ducks',
'Los Angeles Kings',
'New York Rangers',
'Phoenix Coyotes']

In [344]:
playoffs_11_12 = ['Los Angeles Kings',
'New Jersey Devils',
'New York Rangers',
'Phoenix Coyotes',
'Washington Capitals',
'Nashville Predators',
'Philadelphia Flyers',
'St Louis Blues',
'Boston Bruins',
'Ottawa Senators',
'Florida Panthers',
'Pittsburgh Penguins',
'Chicago Blackhawks',
'Detroit Red Wings',
'Vancouver Canucks',
'San Jose Sharks']

In [345]:
playoffs_13_14 = ['Los Angeles Kings',
'New York Rangers',
'Chicago Blackhawks',
'Montreal Canadiens',
'Boston Bruins',
'Anaheim Ducks',
'Pittsburgh Penguins',
'Minnesota Wild',
'Colorado Avalanche',
'Philadelphia Flyers',
'San Jose Sharks',
'Dallas Stars',
'Columbus Blue Jackets',
'St Louis Blues',
'Detroit Red Wings',
'Tampa Bay Lightning']

In [346]:
playoffs_14_15 = ['Chicago Blackhawks',
'Tampa Bay Lightning',
'Anaheim Ducks',
'New York Rangers',
'Washington Capitals',
'Montreal Canadiens',
'Calgary Flames',
'Minnesota Wild',
'New York Islanders',
'Detroit Red Wings',
'Ottawa Senators',
'Nashville Predators',
'Vancouver Canucks',
'St Louis Blues',
'Pittsburgh Penguins',
'Winnipeg Jets']

In [347]:
playoffs_15_16 = ['Pittsburgh Penguins',
'San Jose Sharks',
'Tampa Bay Lightning',
'St Louis Blues',
'Dallas Stars',
'Nashville Predators',
'Washington Capitals',
'New York Islanders',
'Chicago Blackhawks',
'Anaheim Ducks',
'Minnesota Wild',
'Florida Panthers',
'Philadelphia Flyers',
'New York Rangers',
'Detroit Red Wings',
'Los Angeles Kings']

In [348]:
playoffs_16_17 = ['Pittsburgh Penguins',
'Nashville Predators',
'Ottawa Senators',
'Anaheim Ducks',
'Edmonton Oilers',
'Washington Capitals',
'St Louis Blues',
'New York Rangers',
'San Jose Sharks',
'Montreal Canadiens',
'Boston Bruins',
'Toronto Maple Leafs',
'Minnesota Wild',
'Columbus Blue Jackets',
'Calgary Flames',
'Chicago Blackhawks']

In [349]:
playoffs_17_18 = ['Washington Capitals',
'Vegas Golden Knights',
'Tampa Bay Lightning',
'Winnipeg Jets',
'Nashville Predators',
'San Jose Sharks',
'Pittsburgh Penguins',
'Boston Bruins',
'Toronto Maple Leafs',
'Philadelphia Flyers',
'Colorado Avalanche',
'Columbus Blue Jackets',
'Minnesota Wild',
'New Jersey Devils',
'Los Angeles Kings',
'Anaheim Ducks']

In [389]:
# adding playoff data
make_playoff_team(playoffs_07_08, teams_07_08)
make_playoff_team(playoffs_08_09, teams_08_09)
make_playoff_team(playoffs_09_10, teams_09_10)
make_playoff_team(playoffs_10_11, teams_10_11)
make_playoff_team(playoffs_11_12, teams_11_12)
make_playoff_team(playoffs_13_14, teams_13_14)
make_playoff_team(playoffs_14_15, teams_14_15)
make_playoff_team(playoffs_15_16, teams_15_16)
make_playoff_team(playoffs_16_17, teams_16_17)
make_playoff_team(playoffs_17_18, teams_17_18)

In [376]:
# function to drop teams that did not exist during that season?? 

In [390]:
#function to export data to csvs
def to_csv_file(team_df, name):
    path = 'data_cleaned/team_data_by_szn/' + name + '.csv'
    team_df.to_csv(path)

In [392]:
#sending dfs to csvs
to_csv_file(teams_07_08, 'teams_07_08')
to_csv_file(teams_08_09, 'teams_08_09')
to_csv_file(teams_09_10, 'teams_09_10')
to_csv_file(teams_10_11, 'teams_10_11')
to_csv_file(teams_11_12, 'teams_11_12')
to_csv_file(teams_13_14, 'teams_13_14')
to_csv_file(teams_14_15, 'teams_14_15')
to_csv_file(teams_15_16, 'teams_15_16')
to_csv_file(teams_16_17, 'teams_16_17')
to_csv_file(teams_17_18, 'teams_17_18')